# Script for scraping Openinsider for Directors Dealings

Goal of this simple script, is to read in company tickers (and dates), and return all directors dealings of defined types for the period. Data is then cleansed a bit, before being returned as CSVs.

This script is written by Gunnar Sjúrðarson Knudsen at TU Wien on 2022-05-27

Data is used in relevance to courses 330.214 Project & Enterprise Financing, as well as 194.060 Interdisciplinary Project in collaboration with Professor Aussenegg.


### Current Caveats:
* Several companies have different ISINs, but the same ticker. as openinsider (to my knowledge) only works with tickers, this might be misrepresenting in certain cases. **Talk to professor of how to handle this**. Possible solution:
  * ~~Re-read the given excel files, and figure out which ISIN corresponds to which ticker for a given time-period.~~
  * ~~Then split ticker files according to the time periods, and re-save them based on the ISIN (or other identifier)~~
  * Nope... That doesn't work, as tickers aren't unique during periods either... Clueless on how to solve

## Setup

### Load required libraries

In [1]:
# Standard
import pandas as pd
import numpy as np
import pickle

# Time Cleaning
import time
import datetime
from dateutil.relativedelta import relativedelta

# Scraping
import requests
import locale
from pandas.io.json import json_normalize
import io

from os.path import exists

# Cleansing
import pandas as pd
import locale

from os import listdir
from os.path import isfile, join

from IPython.display import clear_output, display

### Define functions

#### Get relevant tickers

In [2]:
def read_tickers_and_isins(_filename):
    clear_output(wait=True)
    print('Reading tickers')
    data = pd.read_excel(_filename, keep_default_na=False, dtype={'TICKER SYMBOL': str})
    tickers = data['TICKER SYMBOL'].str.replace(' ', '+')
    tickers = tickers.astype(str)
    isins = data['ISIN CODE'].str.replace(' ', '+')
    isins = isins.astype(str)
    
    data['TICKER SYMBOL'] = tickers
    data['ISIN CODE'] = isins
    # Save list somewhere for re-reading?
    #display(data)
    return data

#### Define function for downloading a single company

In [3]:
def get_single_directors_dealings(_data_location_insider_raw, _ticker, _from_date, _to_date, _dl_type_string, _dl_date_string):
    #clear_output(wait=True)
    #print('Trying to get directors dealings for ' + _ticker)
    f = _data_location_insider_raw + _ticker + '.csv'
    if not exists(f):
        max_pagination = 100000
        insider_data = pd.read_html(f'http://www.openinsider.com/screener?s={_ticker}&o=&pl=&ph=&ll=&lh=&fd=-1{_dl_date_string}&td=0&tdr=&fdlyl=&fdlyh=&daysago={_dl_type_string}&vl=&vh=&ocl=&och=&sic1=-1&sicl=100&sich=9999&grp=0&nfl=&nfh=&nil=&nih=&nol=&noh=&v2l=&v2h=&oc2l=&oc2h=&sortcol=0&cnt={max_pagination}&page=1')
        insider_data = insider_data[-3]
        insider_data['ticker'] = _ticker
        if insider_data.shape[0] >= max_pagination - 1:
            print(f"POSSIBLE ERROR OCCURED HERE for {_ticker}")

        # Rewrite if nothing is found
        if insider_data.iloc[0, 0] == 'Sort by':
            insider_data = pd.DataFrame(
                columns=['X', 'Filing\xa0Date', 'Trade\xa0Date', 'Ticker', 'Insider Name', 'Title', 'Trade Type',
                         'Price', 'Qty', 'Owned', 'ΔOwn', 'Value', '1d', '1w', '1m', '6m', 'ticker'])
        insider_data.to_csv(_data_location_insider_raw + _ticker + '.csv')
        print(insider_data.shape)

#### Define loops for tickers

In [4]:
def get_all_directors_dealings(_data_location_insider_raw, _data, _download_type):
    # Generate part of URL that defines download type
    dl_type_string = ''
    for t in _download_type:
        dl_type_string = dl_type_string + ('&x' + t.lower() + '=1')
    
    counter = 0
    for index, row in data.iterrows():    
        counter = counter + 1
        ticker = row['TICKER SYMBOL']
        from_date = row['BASE OR ST DATE']
        to_date = row['DATE/TIME']
        clear_output(wait=True)
        print(f'{counter}: Downloading data for ticker {ticker} for period from {from_date} to {to_date}')
        
        # Generate part of URL that defines period
        ## Should be moved to other function - but too late
        if from_date == 'NA':
            from_date = datetime.datetime.now()
        if to_date == 'NA':
            to_date = datetime.datetime.now()
        dl_date_string = '&fdr=' + (f'{from_date.month:02d}') + '%2F' + (f'{from_date.day:02d}') + '%2F' + (f'{from_date.year:04d}') + '+-+' + (f'{to_date.month:02d}') + '%2F' + (f'{to_date.day:02d}') + '%2F' + (f'{to_date.year:04d}')
        
        get_single_directors_dealings(_data_location_insider_raw, ticker, from_date, to_date, dl_type_string, dl_date_string)

#### Define function for cleansing Data

In [5]:
def preprocess_directors_dealings(_raw_location, _preprocessed_location):
    # List of files to process
    filenames = [f for f in listdir(_raw_location) if isfile(join(_raw_location, f))]
    for f in filenames:
        clear_output(wait=True)
        print(f'Processing {f}')

        insider_data = pd.read_csv(_raw_location + f, index_col=0)
        
        # If we got companyName
        if insider_data.shape[1] == 18:
            insider_data.columns = ['X', 'FilingDate', 'TradeDate', 'Ticker', 'CompanyName', 'InsiderName', 'Title', 'TradeType', 'Price', 'Qty', 'Owned', 'delta_Own', 'Value'	, '1d'	, '1w'	, '1m'	, '6m'	, 'ticker']
        # And then rename columns
        if insider_data.shape[1] == 17:
            insider_data.columns = ['X', 'FilingDate', 'TradeDate', 'Ticker', 'InsiderName', 'Title', 'TradeType', 'Price', 'Qty', 'Owned', 'delta_Own', 'Value'	, '1d'	, '1w'	, '1m'	, '6m'	, 'ticker']

        # Process datatypes accordingly
        insider_data['FilingDate'] = pd.to_datetime(insider_data['FilingDate'])
        insider_data['TradeDate'] = pd.to_datetime(insider_data['TradeDate'])
        ## 'Trade\xa0Type' - should this be decoded?
        insider_data['Price'] = insider_data['Price'].astype(str).map(lambda x: x.replace(',', '').strip('+'))
        insider_data['Price'] = insider_data['Price'].map(lambda x: locale.atof(x.strip('$')))
        insider_data['Qty'] = pd.to_numeric(
            (insider_data['Qty']).astype(str).map(lambda x: x.replace(',', '').strip('+')))
        insider_data['Value'] = pd.to_numeric(
            insider_data['Value'].map(lambda x: locale.atof(x.replace(',', '').replace('$', ''))))
        # insider_data['ΔOwn']=insider_data['ΔOwn'].map(lambda x: locale.atof(x.replace(',', '').replace('%','')))

        # Better naming of columns
        insider_data.columns = insider_data.columns.map(lambda x: x.replace('\xa0', '').replace('Δ', 'delta_'))
        insider_data.to_csv(_preprocessed_location + f)


## Scrape data

### Nasdaq Composite

In [6]:
INPUT_FILE = 'source/Nasdaq Composite 2004_2022 - total - Knudsen.xlsx'
#INPUT_FILE = 'source/SmallTestingFile.xlsx'
download_type = ['P', 'S', 'A', 'D', 'G', 'F', 'M', 'X', 'C', 'W']
RAW_DATA_LOCATION = 'raw_download/nasdaq/'
PROCESSED_DATA_LOCATION = 'processed_data/nasdaq/'

# Read in tickers
data = read_tickers_and_isins(INPUT_FILE)
display(data)

# Download the dealings
get_all_directors_dealings(RAW_DATA_LOCATION, data, download_type)

# Cleanse the dealings
preprocess_directors_dealings(RAW_DATA_LOCATION, PROCESSED_DATA_LOCATION)

Processing VANS.csv


### S&P 500

In [7]:
INPUT_FILE = 'source/S&P 500 - Constituents 1989-2022 - Knudsen.xlsx'
#INPUT_FILE = 'source/SmallTestingFile.xlsx'
download_type = ['P', 'S', 'A', 'D', 'G', 'F', 'M', 'X', 'C', 'W']
RAW_DATA_LOCATION = 'raw_download/sop500/'
PROCESSED_DATA_LOCATION = 'processed_data/sop500/'

# Read in tickers
data = read_tickers_and_isins(INPUT_FILE)
display(data)

# Download the dealings
get_all_directors_dealings(RAW_DATA_LOCATION, data, download_type)

# Cleanse the dealings
preprocess_directors_dealings(RAW_DATA_LOCATION, PROCESSED_DATA_LOCATION)

Processing RCL.csv
